In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

In [16]:
import json
import os
# import uuid  # https://docs.python.org/3/library/uuid.html
# import structlog  # for event logging
# # from dotenv import load_dotenv # enviornment vars if we want

# from pygtail import Pygtail
# import boto3
# from minio import Minio
# from dotenv import load_dotenv
from datetime import timedelta, datetime
from time import sleep
from sys import argv
# import threading
# from smart_open import smart_open
import gensim
import gensim.corpora as corpora

# setup to import the preprocessor
import sys
# from botocore.errorfactory import ClientError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce


In [3]:
computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
computing = computing.dropna()
computing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 277
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       263 non-null    object
 1   unit         263 non-null    object
 2   lesson       263 non-null    object
 3   video_title  263 non-null    object
 4   about        263 non-null    object
 5   transcript   263 non-null    object
dtypes: object(6)
memory usage: 14.4+ KB


In [4]:
economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
economics = economics.dropna()
economics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819 entries, 0 to 819
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       819 non-null    object
 1   unit         819 non-null    object
 2   lesson       819 non-null    object
 3   video_title  819 non-null    object
 4   about        819 non-null    object
 5   transcript   819 non-null    object
dtypes: object(6)
memory usage: 44.8+ KB


In [5]:
humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
humanities = humanities.dropna()
humanities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1152
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       1064 non-null   object
 1   unit         1064 non-null   object
 2   lesson       1064 non-null   object
 3   video_title  1064 non-null   object
 4   about        1064 non-null   object
 5   transcript   1064 non-null   object
dtypes: object(6)
memory usage: 58.2+ KB


In [6]:
math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
math = math.dropna()
math.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3351 entries, 0 to 3366
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       3351 non-null   object
 1   unit         3351 non-null   object
 2   lesson       3351 non-null   object
 3   video_title  3351 non-null   object
 4   about        3351 non-null   object
 5   transcript   3351 non-null   object
dtypes: object(6)
memory usage: 183.3+ KB


In [7]:
science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
science = science.dropna()
science.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       2764 non-null   object
 1   unit         2764 non-null   object
 2   lesson       2764 non-null   object
 3   video_title  2764 non-null   object
 4   about        2764 non-null   object
 5   transcript   2764 non-null   object
dtypes: object(6)
memory usage: 151.2+ KB


In [8]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
validation = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
validation = validation.dropna()
validation.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [15]:
computing['course'].value_counts()

Pixar in a Box                              175
Computer science                             48
Code.org                                     13
Computer programming                         12
Computers and the Internet                    8
AP®︎/College Computer Science Principles      7
Name: course, dtype: int64

In [11]:
economics['course'].value_counts()

Finance and capital markets    317
Macroeconomics                 172
Microeconomics                 139
AP®︎/College Macroeconomics    105
AP®︎/College Microeconomics     86
Name: course, dtype: int64

In [12]:
humanities['course'].value_counts()

World history                                     185
AP®︎/College Art History                          169
US government and civics                          139
AP®︎/College US History                           133
US history                                        127
AP®︎/College US Government and Politics            87
Storytelling                                       69
Big History Project                                60
World History Project - Origins to the Present     56
World History Project - 1750 to the Present        39
Name: course, dtype: int64

In [13]:
math['course'].value_counts()

Algebra (all content)         914
Geometry (all content)        345
Statistics and probability    314
Precalculus                   287
Differential Calculus         252
College Algebra               237
Integral Calculus             232
AP®︎/College Statistics       214
Multivariable calculus        176
Linear algebra                136
High school statistics         98
Trigonometry                   76
Differential equations         70
Name: course, dtype: int64

In [14]:
science['course'].value_counts()

Chemistry library                               389
Physics library                                 386
Biology library                                 359
Organic chemistry                               309
AP®︎/College Chemistry                          211
AP®︎/College Biology                            200
Electrical engineering                          198
AP®︎/College Physics 2                          129
High school biology                             116
High school physics                             111
Cosmology and astronomy                          89
AP®︎/College Physics 1                           61
AP®︎/College Environmental science               49
High school biology - NGSS                       43
Middle school biology - NGSS                     39
High school physics  - NGSS                      35
Middle school physics - NGSS                     21
Middle school Earth and space science - NGSS     19
Name: course, dtype: int64

In [21]:
khan_dfs = [computing, economics, humanities, math, science]
khan = pd.concat(khan_dfs, axis=0)
khan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8261 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       8261 non-null   object
 1   unit         8261 non-null   object
 2   lesson       8261 non-null   object
 3   video_title  8261 non-null   object
 4   about        8261 non-null   object
 5   transcript   8261 non-null   object
dtypes: object(6)
memory usage: 451.8+ KB


In [44]:
# remap the courses to more broad categories: https://stackoverflow.com/a/16476974
labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'macroeconomics', 'microeconomics']
for lbl in labels:
    for index, row in khan.iterrows():
        if lbl in row['course'].lower():
            row['course'] = lbl
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
geometry                                         345
finance and capital markets                      317
macroeconomics                                   277
microeconomics                                   225
electrical engineering                           198
pixar in a box                                   175
us government and civics                         139
cosmology and astronomy                           89
ap®︎/college us government and politics           87
trigonometry                                      76
differential equations                            70
storytelling                                  